#**download data directly from Kaggle**

In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

 89% 18.0M/20.1M [00:01<00:00, 13.7MB/s]
100% 20.1M/20.1M [00:01<00:00, 10.8MB/s]


#**Unzip**


In [3]:
!unzip udea*.zip > /dev/null

In [ ]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  50135751 test.csv
   692501  10666231 118025055 train.csv
  1286075  15528571 172877479 total


## load `train.csv` data with pandas



In [4]:
import pandas as pd
import numpy as np

z = pd.read_csv("train.csv")
print ("shape of loaded dataframe", z.shape)


shape of loaded dataframe (692500, 12)


In [5]:
data = z.copy()

#Fase de preprocesado


En esta sección de códigos que siguen se van a realizar las distintas transformaciones necesarias para las variables de interés que se utilizarán en el modelo predictivo, todo con el objetivo de poder trabajar, medirlas y estudiarlas para permitirnos acercarnos a predicciones con un buen porcentaje de acierto en nuestro modelo.

##Cleaning VALORMATRICULA column



 Este proceso convierte los rangos de matrícula en valores numéricos usando un diccionario, lo que facilita el análisis posterior. Primero, se mapean los rangos textuales de matrícula a valores promedio definidos. Luego para los valores nulos, se calcula la media y desviación estándar para generar datos simulados que siguen una distribución similar. Finalmente, estos valores generados reemplazan los datos faltantes en la columna, dejando una columna de matrícula completa y coherente para análisis.

In [6]:
def cleaning_val_matricula(data):
    vals_matricula = {
        'Entre 5.5 millones y menos de 7 millones': 6.25,
        'Entre 2.5 millones y menos de 4 millones': 3.25,
        'Entre 4 millones y menos de 5.5 millones': 4.75,
        'Más de 7 millones': 7.0,
        'Entre 1 millón y menos de 2.5 millones': 1.75,
        'Entre 500 mil y menos de 1 millón': 0.75,
        'Menos de 500 mil': 0.25,
        'No pagó matrícula': 0,
    }
    data['ESTU_VALORMATRICULAUNIVERSIDAD'] = data['ESTU_VALORMATRICULAUNIVERSIDAD'].map(vals_matricula)

    # Se calcula la media y desviación estándar
    mean = data['ESTU_VALORMATRICULAUNIVERSIDAD'].mean()
    std = data['ESTU_VALORMATRICULAUNIVERSIDAD'].std()

    #tamaño de la muestra (valores nulos)
    sample = np.random.normal(mean, std, data['ESTU_VALORMATRICULAUNIVERSIDAD'].isna().sum())
    data.loc[data['ESTU_VALORMATRICULAUNIVERSIDAD'].isna(), 'ESTU_VALORMATRICULAUNIVERSIDAD'] = sample
    has_nan = data['ESTU_VALORMATRICULAUNIVERSIDAD'].isnull().any()
    print(has_nan)  # True if there are NaNs, otherwise False

## Cleaning FAMI_TIENEINTERNET column


Simplemente mapeamos el dataset con tres reglas para asignar valores númericos a esta columna

In [7]:
def cleaning_internet(data):
    data['FAMI_TIENEINTERNET'] = data['FAMI_TIENEINTERNET'].fillna('Sin Información')
    data['FAMI_TIENEINTERNET'] = data['FAMI_TIENEINTERNET'].map({
        'Si': 1,
        'No': 0,
        'Sin Información': -1
    })
    has_nan = data['FAMI_TIENEINTERNET'].isnull().any()
    print(has_nan)  # True if there are NaNs, otherwise False


##Cleaning rendimiento global


Este código convierte los niveles de rendimiento de texto en valores numéricos para facilitar el análisis. Mediante un diccionario, mapea los valores de "bajo", "medio-bajo", "medio-alto" y "alto" a los números 0, 1, 2 y 3, respectivamente. Luego, reemplaza en la columna "RENDIMIENTO_GLOBAL" los valores de texto originales con estos valores numéricos.

In [8]:
def cleaning_rendimiento(data):
  values = {"bajo":0,"medio-bajo":1, "medio-alto":2, "alto":3 }
  data["RENDIMIENTO_GLOBAL"] = data["RENDIMIENTO_GLOBAL"].map(values)
  has_nan = data['RENDIMIENTO_GLOBAL'].isnull().any()
  print(has_nan)  # True if there are NaNs, otherwise False

## Cleaning FAMI_EDUCACIONPADRE column

Primero, rellena valores nulos con “Sin Información” y asigna -1 a categorías como “No Aplica”, “Sin Información” y “No sabe” para indicar la ausencia de datos. Luego, utiliza un orden específico para los niveles educativos y aplica OrdinalEncoder para transformar estos niveles en valores numéricos según el orden predefinido, asegurando que sólo se codifiquen las filas con información válida.

In [9]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
def cleaning_padre(data):
    data['FAMI_EDUCACIONPADRE'] = data['FAMI_EDUCACIONPADRE'].fillna('Sin Información')
    data['FAMI_EDUCACIONPADRE'] = data['FAMI_EDUCACIONPADRE'].replace({
        'No Aplica': -1, 'Sin Información': -1, 'No sabe': -1
    })
    options = ['Ninguno', 'Primaria incompleta', 'Primaria completa',
              'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa',
              'Técnica o tecnológica incompleta', 'Técnica o tecnológica completa',
              'Educación profesional incompleta', 'Educación profesional completa',
              'Postgrado']
    mask = data['FAMI_EDUCACIONPADRE'] != -1
    encoder = OrdinalEncoder(categories=[options])
    data.loc[mask, 'FAMI_EDUCACIONPADRE'] = encoder.fit_transform(data.loc[mask, ['FAMI_EDUCACIONPADRE']])
    data['FAMI_EDUCACIONPADRE'] = data['FAMI_EDUCACIONPADRE'].astype(int)
    has_nan = data['FAMI_EDUCACIONPADRE'].isnull().any()
    print(has_nan)  # True if there are NaNs, otherwise False

##cleaning FAMI_EDUCACIONMADRE

misma limpieza anterior pero para la madre

In [10]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
def cleaning_madre(data):
    data['FAMI_EDUCACIONMADRE'] = data['FAMI_EDUCACIONMADRE'].fillna('Sin Información')
    data['FAMI_EDUCACIONMADRE'] = data['FAMI_EDUCACIONMADRE'].replace({
        'No Aplica': -1, 'Sin Información': -1, 'No sabe': -1
    })
    options = ['Ninguno', 'Primaria incompleta', 'Primaria completa',
              'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa',
              'Técnica o tecnológica incompleta', 'Técnica o tecnológica completa',
              'Educación profesional incompleta', 'Educación profesional completa',
              'Postgrado']
    mask = data['FAMI_EDUCACIONMADRE'] != -1
    encoder = OrdinalEncoder(categories=[options])
    data.loc[mask, 'FAMI_EDUCACIONMADRE'] = encoder.fit_transform(data.loc[mask, ['FAMI_EDUCACIONMADRE']])
    data['FAMI_EDUCACIONMADRE'] = data['FAMI_EDUCACIONMADRE'].astype(int)
    has_nan = data['FAMI_EDUCACIONMADRE'].isnull().any()
    print(has_nan)  # True if there are NaNs, otherwise False

##ESTU_PRGM_ACADEMICO

Este código agrupa programas académicos en categorías utilizando aprendizaje automático. Primero, convierte los nombres de los programas en representaciones numéricas (embeddings) mediante el modelo SentenceTransformer. Luego, aplica el algoritmo de K-Means para crear grupos o categorías de programas académicos, donde cada grupo representa una categoría temática.

In [12]:
pip install sentence-transformers scikit-learn


In [13]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

def cleaning_programa(data):
    # Lista de programs académicos
    programs = data['ESTU_PRGM_ACADEMICO'].unique()

    # Cargar el modelo de embeddings
    model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

    # Generar embeddings para cada programa académico
    embeddings = model.encode(programs)

    # Elegir el número de clusters
    num_clusters = 15

    # Aplicar K-Means para agrupar
    kmeans = KMeans(n_clusters=num_clusters, random_state=0)
    labels = kmeans.fit_predict(embeddings)

    # Crear un diccionario de mapeo de programs a categorías (como números enteros)
    programa_a_categoria = {programa: label + 1 for programa, label in zip(programs, labels)}

    # Reemplazar los nombres de programs en el DataFrame por sus categorías numéricas
    data['ESTU_PRGM_ACADEMICO'] = data['ESTU_PRGM_ACADEMICO'].map(programa_a_categoria)
    has_nan = data['ESTU_PRGM_ACADEMICO'].isnull().any()
    print(has_nan)  # True if there are NaNs, otherwise False


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


##ESTU_HORASSEMANATRABAJA

In [14]:
import pandas as pd
import numpy as np

def cleaning_horas(data):
    # Mapeo de rangos a valores numéricos aproximados
    rango_a_horas = {
        'Menos de 10 horas': 5,        # Promedio del rango (0-10 horas)
        'Entre 11 y 20 horas': 15,     # Promedio del rango (11-20 horas)
        'Entre 21 y 30 horas': 25,     # Promedio del rango (21-30 horas)
        'Más de 30 horas': 35,         # Se puede usar un promedio o un valor base
        '0': 0                         # Indicador de que no trabaja
    }
    print(data['ESTU_HORASSEMANATRABAJA'].unique())
    # Reemplazar los valores con el mapeo definido
    data['ESTU_HORASSEMANATRABAJA'] = data['ESTU_HORASSEMANATRABAJA'].replace(rango_a_horas)

    # Convertir la columna a numérico
    data['ESTU_HORASSEMANATRABAJA'] = pd.to_numeric(data['ESTU_HORASSEMANATRABAJA'], errors='coerce')

    # Imputar valores NaN usando una distribución normal basada en los datos existentes (sin contar el 0)
    mean = data.loc[data['ESTU_HORASSEMANATRABAJA'] > 0, 'ESTU_HORASSEMANATRABAJA'].mean()
    std = data.loc[data['ESTU_HORASSEMANATRABAJA'] > 0, 'ESTU_HORASSEMANATRABAJA'].std()
    num_nan = data['ESTU_HORASSEMANATRABAJA'].isna().sum()

    # Generar valores aleatorios solo para los NaN, manteniendo los '0' como "no trabaja"
    data.loc[data['ESTU_HORASSEMANATRABAJA'].isna(), 'ESTU_HORASSEMANATRABAJA'] = np.random.normal(mean, std, num_nan)

    print(data['ESTU_HORASSEMANATRABAJA'].unique())  # Para verificar los resultados
    has_nan = data['ESTU_HORASSEMANATRABAJA'].isnull().any()
    print(has_nan)  # True if there are NaNs, otherwise False



## Cleaning de ESTU_PRGM_DEPARTAMENTO

In [15]:
from sklearn.preprocessing import LabelEncoder
def department_cleaning(data):
  # Crear un LabelEncoder
  le = LabelEncoder()
  # Ajustar y transformar la columna `ESTU_PRGM_DEPARTAMENTO`
  data['ESTU_PRGM_DEPARTAMENTO'] = le.fit_transform(data['ESTU_PRGM_DEPARTAMENTO'])
  has_nan = data['ESTU_PRGM_DEPARTAMENTO'].isnull().any()
  print(has_nan)  # True if there are NaNs, otherwise False

##Cleaning FAMI_ESTRATOVIVIENDA

In [16]:
def cleaning_estrato(data):
  # data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].fillna(-1)
  maping ={
      'Estrato 1': 1,
      'Estrato 2': 2,
      'Estrato 3': 3,
      'Estrato 4': 4,
      'Estrato 5': 5,
      'Estrato 6': 6,
      'Sin Estrato': -1
  }
  data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].map(maping)
  data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].fillna(-1)
  has_nan = data['FAMI_ESTRATOVIVIENDA'].isnull().any()
  print(has_nan)  # True if there are NaNs, otherwise False

## Cleaning ESTU_PAGOMATRICULAPROPIO

In [17]:
def cleaning_pagomatriculapropio(data):
    # Fill NaNs with -1 first
    data['ESTU_PAGOMATRICULAPROPIO'] = data['ESTU_PAGOMATRICULAPROPIO'].fillna(-1)

    # Map 'Si' to 1 and 'No' to 0
    data['ESTU_PAGOMATRICULAPROPIO'] = data['ESTU_PAGOMATRICULAPROPIO'].map({
        'Si': 1,
        'No': 0,
    }).fillna(-1).astype(int)  # Convert any remaining NaNs to -1

    print(data['ESTU_PAGOMATRICULAPROPIO'].unique())

In [18]:
cleaning_val_matricula(data)
cleaning_internet(data)
cleaning_rendimiento(data)
cleaning_padre(data)
cleaning_madre(data)
cleaning_programa(data)
cleaning_horas(data)
department_cleaning(data)
cleaning_estrato(data)
cleaning_pagomatriculapropio(data)

False
False
False
False
False


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

False
['Menos de 10 horas' '0' 'Más de 30 horas' 'Entre 21 y 30 horas'
 'Entre 11 y 20 horas' nan]


<ipython-input-14-5cc7fc740a4d>:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['ESTU_HORASSEMANATRABAJA'] = data['ESTU_HORASSEMANATRABAJA'].replace(rango_a_horas)


[ 5.          0.         35.         ... 11.45437427 29.12788504
 36.88447411]
False
False
False
[ 0  1 -1]


# Análisis y exclusion de las variables


De momento vemos que gráficamente las variables contribuyen y tienen una distribución que puede ser importante en las predicciones por lo que sólo vamos a soltar la columna Id.

In [19]:
data.drop(columns=['ID'], inplace=True)
# data.drop(columns=['Categoria'], inplace=True)
data.head()


,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,20212,5,4,6.25,5.0,3.0,1,5,9,0,2
1,20212,5,3,3.25,0.0,3.0,0,6,5,0,0
2,20203,1,4,3.25,35.0,3.0,1,4,4,0,0
3,20195,10,26,4.75,0.0,4.0,1,-1,4,0,3
4,20212,5,1,3.25,25.0,3.0,1,2,2,0,1


In [ ]:
data.shape

(692500, 11)

# Modelo

Primero declaramos la columna de interés

In [20]:
y_col = 'RENDIMIENTO_GLOBAL'

rmap = {'alto': 3, 'bajo':0, 'medio-bajo':1, 'medio-alto':2}
# z[y_col] = [rmap[i] for i in z[y_col]]
# z.head()

## Construimos X y Y para el modelo de entrenamiento


In [21]:
data = data[sorted(data.columns)] #Las ordenamos

X = data[[c for c in data.columns if c!=y_col]].values # En x vamos a tomar los que no son la columna de interés
y = data[y_col].values # En y vamos a tomar la columna de interés
X.shape, y.shape # Tamaño de X y Y

((692500, 10), (692500,))

## Ahora vamos a dividir el modelo en la fase de entrenamiento y testeo

Este paso es escencial porque así vamos a poder entrenar el modelo con el dataset y probar la precisión con un dataset que el modelo en si no ha interactuado nunca

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
Xtr, Xts, ytr, yts = train_test_split(X,y, train_size=0.8)
Xtr.shape, Xts.shape, ytr.shape, yts.shape

((554000, 10), (138500, 10), (554000,), (138500,))

estamos tomando el 80% de los datos del dataset para realizar el entrenamiento

#lightgbm model

In [24]:
pip install dask[dataframe]

In [25]:
from lightgbm import LGBMClassifier
lgb_model = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
lgb_model.fit(Xtr, ytr)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.093381 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 602
[LightGBM] [Info] Number of data points in the train set: 554000, number of used features: 10
[LightGBM] [Info] Start training from score -1.386879
[LightGBM] [Info] Start training from score -1.391107
[LightGBM] [Info] Start training from score -1.394341
[LightGBM] [Info] Start training from score -1.372983


LGBMClassifier(max_depth=6, random_state=42)

obtener predicciones

In [28]:
preds_tr = lgb_model.predict(Xtr)
preds_ts = lgb_model.predict(Xts)
np.mean(preds_tr==ytr), np.mean(preds_ts==yts)

(0.42301083032490977, 0.4188158844765343)

## Loading and cleaning the test.csv file


In [26]:
zt = pd.read_csv("test.csv")
zt.drop(columns=['Unnamed: 0'], inplace=True)
zt

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si
1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No
2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No
3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No
4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No
...,...,...,...,...,...,...,...,...,...,...,...
296781,496981,20195,ADMINISTRACION DE EMPRESAS,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 1,Si,Primaria incompleta,Primaria incompleta,Si
296782,209415,20183,DERECHO,META,Entre 1 millón y menos de 2.5 millones,0,Estrato 4,Si,Educación profesional completa,Educación profesional completa,No
296783,239074,20212,DERECHO,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Educación profesional completa,No
296784,963852,20195,INGENIERIA AERONAUTICA,ANTIOQUIA,Entre 5.5 millones y menos de 7 millones,Entre 11 y 20 horas,Estrato 3,Si,Educación profesional completa,Educación profesional completa,No


In [27]:
zt.shape

(296786, 11)

In [ ]:
zt_ids = zt['ID'].values
zt.drop(columns=['ID'], inplace=True)
zt.head()

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si
1,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No
2,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No
3,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No
4,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No


Cleaning functions

In [ ]:
cleaning_val_matricula(zt)
cleaning_internet(zt)
cleaning_padre(zt)
cleaning_madre(zt)
cleaning_programa(zt)
cleaning_horas(zt)
department_cleaning(zt)
cleaning_estrato(zt)
cleaning_pagomatriculapropio(zt)
zt.head()

False
False
False
False


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


False
['Menos de 10 horas' 'Entre 21 y 30 horas' '0' 'Entre 11 y 20 horas'
 'Más de 30 horas' nan]
[ 5.         25.          0.         ... 13.85102358 17.633488
 32.0491543 ]
False
False
False
[ 1  0 -1]


<ipython-input-20-5cc7fc740a4d>:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['ESTU_HORASSEMANATRABAJA'] = data['ESTU_HORASSEMANATRABAJA'].replace(rango_a_horas)


,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,20183,11,5,0.25,5.0,3.0,1,6,2,1
1,20203,15,1,3.25,25.0,2.0,1,4,6,0
2,20212,10,4,1.75,0.0,3.0,1,3,4,0
3,20195,14,27,1.75,25.0,1.0,0,1,1,0
4,20212,10,3,3.25,15.0,2.0,1,4,4,0


Creamos el array X para predecir con base en zt

In [ ]:
X_test_data = zt[sorted(zt.columns)].values
X_test_data.shape

(296786, 10)

Invocación del modelo

In [ ]:
preds_test_data = lgb_model.predict(X_test_data)

## Subir predicciones a kaggle

In [ ]:
# inverse mapping
rmapi = {v:k for k,v in rmap.items()}
text_preds_test_data = [rmapi[i] for i in preds_test_data]


In [ ]:
# create dataframe

submission = pd.DataFrame([zt_ids, text_preds_test_data], index=['ID', 'RENDIMIENTO_GLOBAL']).T
submission

,ID,RENDIMIENTO_GLOBAL
0,550236,alto
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo
...,...,...
296781,496981,medio-bajo
296782,209415,alto
296783,239074,medio-alto
296784,963852,alto


In [ ]:
# save to file ready to submit
submission.to_csv("my_submission.csv", index=False)

In [ ]:
!head my_submission.csv

ID,RENDIMIENTO_GLOBAL
550236,alto
98545,medio-alto
499179,alto
782980,bajo
785185,bajo
58495,medio-bajo
705444,alto
557548,alto
519909,bajo


In [ ]:
submission.shape

(296786, 2)

In [ ]:
!kaggle competitions submit -c udea-ai4eng-20242 -f my_submission.csv -m "Miguel Serna submission with random forest model"

100% 4.04M/4.04M [00:00<00:00, 16.0MB/s]
Successfully submitted to UDEA/ai4eng 20242 - Pruebas Saber Pro Colombia